In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np

In [ ]:
# Definir las dimensiones de las imágenes
img_width, img_height = 150, 150

# Establecer la ruta de los datos de entrenamiento y validación
train_data_dir = r'C:\Users\HP\Documents\Pycham\pruebafinal\CarneDataset\train'
validation_data_dir = r'C:\Users\HP\Documents\Pycham\pruebafinal\CarneDataset\test'

# Asegúrate de ajustar estos valores a la cantidad de imágenes que tienes
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# Cargar VGG16 sin la capa superior (fully-connected layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Agregar tus propias capas
x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # Agregando dropout
predictions = Dense(8, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Preprocesar las imágenes
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)  # Aumento de datos

test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)


In [14]:

# Generar las predicciones para el conjunto de entrenamiento
train_steps = nb_train_samples // batch_size
train_predictions = model.predict_generator(train_generator, steps=train_steps)
train_predictions = np.argmax(train_predictions, axis=1)

# Generar las predicciones para el conjunto de prueba
validation_steps = nb_validation_samples // batch_size
validation_predictions = model.predict_generator(validation_generator, steps=validation_steps)
validation_predictions = np.argmax(validation_predictions, axis=1)

# Calcular las matrices de confusión
train_confusion_matrix = confusion_matrix(train_generator.classes[:len(train_predictions)], train_predictions)
validation_confusion_matrix = confusion_matrix(validation_generator.classes[:len(validation_predictions)], validation_predictions)

print('Matriz de confusión del entrenamiento:')
print(train_confusion_matrix)
print('Matriz de confusión del test:')
print(validation_confusion_matrix)

# Calcular precision, recall y f1-score para el conjunto de entrenamiento
train_precision = precision_score(train_generator.classes[:len(train_predictions)], train_predictions, average='weighted')
train_recall = recall_score(train_generator.classes[:len(train_predictions)], train_predictions, average='weighted')
train_f1_score = f1_score(train_generator.classes[:len(train_predictions)], train_predictions, average='weighted')

print('\nMétricas del entrenamiento:')
print(f'Precision: {train_precision}')
print(f'Recall: {train_recall}')
print(f'F1 score: {train_f1_score}')

# Calcular precision, recall y f1-score para el conjunto de prueba
validation_precision = precision_score(validation_generator.classes[:len(validation_predictions)], validation_predictions, average='weighted')
validation_recall = recall_score(validation_generator.classes[:len(validation_predictions)], validation_predictions, average='weighted')
validation_f1_score = f1_score(validation_generator.classes[:len(validation_predictions)], validation_predictions, average='weighted')

print('\nMétricas del test:')
print(f'Precision: {validation_precision}')
print(f'Recall: {validation_recall}')
print(f'F1 score: {validation_f1_score}')


C:\Users\HP\AppData\Local\Temp\ipykernel_15412\2801828989.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  train_predictions = model.predict_generator(train_generator, steps=train_steps)


C:\Users\HP\AppData\Local\Temp\ipykernel_15412\2801828989.py:11: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  validation_predictions = model.predict_generator(validation_generator, steps=validation_steps)


Matriz de confusión del entrenamiento:
[[  0   8   3  46   1   4   0]
 [  0  28   7 144   3  30   1]
 [  0  17   5  68   1  14   0]
 [  0 152  40 607  13 136   1]
 [  0   4   4  27   0   2   0]
 [  0  30  10 132   6  25   1]
 [  0   5   2  45   2   9   0]]
Matriz de confusión del test:
[[  0   0   0   0   0   0   1   0]
 [  0   0  12   2  22   0  12   0]
 [  0   0  12   3  68   0  14   0]
 [  0   0   3   2  33   1   5   1]
 [  0   0  54  20 294   7  82   2]
 [  0   0   2   0  12   1   4   0]
 [  0   0  10   5  72   1  25   1]
 [  0   0   4   1   9   1   2   0]]

Métricas del entrenamiento:
Precision: 0.36367443979360364
Recall: 0.40722596448254744
F1 score: 0.38397307141028536

Métricas del test:
Precision: 0.37588714733542317
Recall: 0.4175
F1 score: 0.39513549596444336


C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
